In [ ]:
#@title **设置参数/Parameters**
language = "ja" # @param {type:"string"}
huggingface_token = "hf_" # @param {type:"string"}
#@markdown 请在上方输入你的 Hugging Face 令牌 (Enter your Hugging Face token above)

# 导入所需的所有模块
import os
import shutil
import logging
from google.colab import drive
from google.colab import files
from google.colab import output
from IPython.display import clear_output, display
import geemap
from ipytree import Tree, Node
import ipywidgets as widgets
from ipywidgets import interactive
import torch

# 验证令牌不为空
if not huggingface_token:
    raise ValueError("请输入 Hugging Face 令牌！(Please enter your Hugging Face token!)")

In [ ]:
#@title **从谷歌网盘选择文件/Select File From Google Drive**

# @markdown <font size="2">Navigate to the file you want to transcribe, left-click to highlight the file, then click 'Select' button to confirm.
# @markdown <br/>从网盘目录中选择要转换的文件(视频/音频），单击选中文件，点击'Select'按钮以确认。</font><br/>
# @markdown <br/><font size="2">If use local file, ignore this cell and move to the next.
# @markdown <br/>若希望从本地上传文件，则跳过此步执行下一单元格。</font><br/>
# @markdown <br/><font size="2">If file uploaded to drive after execution, execute this cell again to refresh.
# @markdown <br/>若到这一步才上传文件到谷歌盘，则重复执行本单元格以刷新文件列表。</font>
!pip install geemap
from google.colab import drive
from google.colab import files
import os
import logging
from IPython.display import clear_output
import geemap

clear_output()
drive.mount('/drive')

print('Google Drive is mounted，please select file')
print('谷歌云盘挂载完毕，请选择要转换的文件')

from ipytree import Tree, Node
import ipywidgets as widgets
from ipywidgets import interactive
from google.colab import output
output.enable_custom_widget_manager()
use_drive = True
global drive_dir
drive_dir = []

def file_tree():
    # create widgets as a simple file browser
    full_widget = widgets.HBox()
    left_widget = widgets.VBox()
    right_widget = widgets.VBox()

    path_widget = widgets.Text()
    path_widget.layout.min_width = '300px'
    select_widget = widgets.Button(
      description='Select', button_style='primary', tooltip='Select current media file.'
      )
    drive_url = widgets.Output()

    right_widget.children = [select_widget]
    full_widget.children = [left_widget]

    tree_widget = widgets.Output()
    tree_widget.layout.max_width = '300px'
    tree_widget.overflow = 'auto'

    left_widget.children = [path_widget,tree_widget]

    # init file tree
    my_tree = Tree(multiple_selection=False)
    my_tree_dict = {}
    media_names = []

    def select_file(b):
        drive_dir.append(path_widget.value)
        print('File selected，please continue to select more or execute next cell')
        print('已选择文件，可以继续选择或执行下个单元格')

    select_widget.on_click(select_file)

    def handle_file_click(event):
        if event['new']:
            cur_node = event['owner']
            for key in my_tree_dict.keys():
                if (cur_node is my_tree_dict[key]) and (os.path.isfile(key)):
                    try:
                        with open(key) as f:
                            path_widget.value = key
                            path_widget.disabled = False
                            select_widget.disabled = False
                            full_widget.children = [left_widget, right_widget]
                    except Exception as e:
                        path_widget.value = key
                        path_widget.disabled = True
                        select_widget.disabled = True
                        return

    def handle_folder_click(event):
        if event['new']:
            full_widget.children = [left_widget]

    # redirect cwd to default drive root path and add nodes
    my_dir = '/drive/MyDrive'
    my_root_name = my_dir.split('/')[-1]
    my_root_node = Node(my_root_name)
    my_tree_dict[my_dir] = my_root_node
    my_tree.add_node(my_root_node)
    my_root_node.observe(handle_folder_click, 'selected')

    for root, d_names, f_names in os.walk(my_dir):
        folders = root.split('/')
        for folder in folders:
            if folder.startswith('.'):
                continue
        for d_name in d_names:
            if d_name.startswith('.'):
                d_names.remove(d_name)
        for f_name in f_names:
            if f_name.lower().endswith(('mp3','m4a','flac','aac','wav','mp4','mkv','ts','flv')):
                media_names.append(f_name)

        d_names.sort()
        f_names.sort()
        media_names.sort()
        keys = my_tree_dict.keys()

        if root not in my_tree_dict.keys():
          name = root.split('/')[-1]
          dir_name = os.path.dirname(root)
          parent_node = my_tree_dict[dir_name]
          node = Node(name)
          my_tree_dict[root] = node
          parent_node.add_node(node)
          node.observe(handle_folder_click, 'selected')

        if len(media_names) > 0:
              parent_node = my_tree_dict[root]
              parent_node.opened = False
              for f_name in media_names:
                  node = Node(f_name)
                  node.icon = 'file'
                  full_path = os.path.join(root, f_name)
                  my_tree_dict[full_path] = node
                  parent_node.add_node(node)
                  node.observe(handle_file_click, 'selected')
        media_names.clear()

    with tree_widget:
      tree_widget.clear_output()
      display(my_tree)

    return full_widget

# 显示文件选择器
tree = file_tree()
display(tree)

Mounted at /drive
Google Drive is mounted，please select file
谷歌云盘挂载完毕，请选择要转换的文件


File selected，please continue to select more or execute next cell
已选择文件，可以继续选择或执行下个单元格


In [ ]:
#@title **开始语音转录/Start voice transcription**
# 1. 安装必要的依赖
!pip uninstall -y whisperx faster-whisper
!pip install faster-whisper==0.9.0
!pip install git+https://github.com/m-bain/whisperX.git

# 2. 设置环境变量
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ["HUGGING_FACE_HUB_TOKEN"] = huggingface_token

# 4. 处理输入文件
if len(drive_dir) > 0:
    input_file = drive_dir[-1]
    new_filename = "input_audio.wav"
    shutil.copy(input_file, new_filename)
    print(f"Copied {input_file} to {new_filename}")
else:
    uploaded = files.upload()
    uploaded_filename = list(uploaded.keys())[0]
    new_filename = "input_audio.wav"
    shutil.move(uploaded_filename, new_filename)
    print(f"Renamed {uploaded_filename} to {new_filename}")

# 5. 创建输出目录
!mkdir -p output

# 6. 检查 GPU 是否可用
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
print(f"PyTorch version: {torch.__version__}")

# 7. 运行 WhisperX
!whisperx input_audio.wav \
    --model large-v2 \
    --compute_type float16 \
    --output_dir output \
    --output_format srt \
    --language {language} \
    --hf_token $HUGGING_FACE_HUB_TOKEN \
    --device cuda

# 8. 查看输出结果
!ls -l output/
!cat output/*.srt

Found existing installation: whisperx 3.2.0
Uninstalling whisperx-3.2.0:
  Successfully uninstalled whisperx-3.2.0
Found existing installation: faster-whisper 1.1.0
Uninstalling faster-whisper-1.1.0:
  Successfully uninstalled faster-whisper-1.1.0
  Using cached faster_whisper-0.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached av-10.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached ctranslate2-3.24.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
Using cached faster_whisper-0.9.0-py3-none-any.whl (1.5 MB)
Using cached av-10.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.0 MB)
Using cached ctranslate2-3.24.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.8 MB)
Using cached tokenizers-0.14.1-cp31